In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
!pip install transformers
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Set the path for the output file
path = '/content/output_file.csv'

# Load the dataset from the CSV file
data = pd.read_csv(path)

# Split the dataset into training and testing sets
X = data['text']
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize and encode the training data
X_train_encodings = tokenizer.batch_encode_plus(
    X_train.tolist(),
    truncation=True,
    padding=True,
    max_length=256,
    return_tensors='pt'
)

# Tokenize and encode the testing data
X_test_encodings = tokenizer.batch_encode_plus(
    X_test.tolist(),
    truncation=True,
    padding=True,
    max_length=256,
    return_tensors='pt'
)





Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Create the BERT model
model = BertModel.from_pretrained('bert-base-uncased')

# Define the sentiment classification model
class SentimentClassifier(nn.Module):
    def __init__(self, bert_model):
        super(SentimentClassifier, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(768, 1)

    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        output = self.dropout(pooled_output)
        output = self.fc(output)
        return output

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Initialize the sentiment classification model
classifier = SentimentClassifier(model)

# Set the device for training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
classifier.to(device)

# Convert the data to tensors and move to the device
X_train_input_ids = X_train_encodings['input_ids'].to(device)
X_train_attention_mask = X_train_encodings['attention_mask'].to(device)
y_train = torch.tensor(y_train.values, dtype=torch.float).unsqueeze(1).to(device)

X_test_input_ids = X_test_encodings['input_ids'].to(device)
X_test_attention_mask = X_test_encodings['attention_mask'].to(device)
y_test = torch.tensor(y_test.values, dtype=torch.float).unsqueeze(1).to(device)

# Define the optimizer and loss function
optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-5)
criterion = nn.BCEWithLogitsLoss()

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    classifier.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = classifier(X_train_input_ids, X_train_attention_mask)
    loss = criterion(outputs, y_train)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    # Evaluation on the testing set
    classifier.eval()
    with torch.no_grad():
        test_outputs = classifier(X_test_input_ids, X_test_attention_mask)
        test_predictions = torch.round(torch.sigmoid(test_outputs))

    accuracy = accuracy_score(y_test.cpu().numpy(), test_predictions.cpu().numpy())
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}, Accuracy: {accuracy}')

# Final evaluation on the testing set
classifier.eval()
with torch.no_grad():
    test_outputs = classifier(X_test_input_ids, X_test_attention_mask)
    test_predictions = torch.round(torch.sigmoid(test_outputs))

accuracy = accuracy_score(y_test.cpu().numpy(), test_predictions.cpu().numpy())
print(f'Final Accuracy: {accuracy}')
